In [1]:
import time
import os
import json
import whisperx
import gc
from whisperx.diarize import DiarizationPipeline
start_time = time.time()
device = "cpu" # "cuda"
audio_file = "audio.wav"
batch_size = 16 # reduce if low on GPU mem
compute_type = "int8" # "float16" # change to "int8" if low on GPU mem (may reduce accuracy)


/home/slava/Documents/projects/Hanzo/wisper_python31013/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/slava/Documents/projects/Hanzo/wisper_python31013/.venv/lib/python3.10/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


In [27]:

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("large-v3", device, compute_type=compute_type, download_root="./models")
# save model to local path (optional)
# model_dir = "/path/"
# model = whisperx.load_model("large-v2", device, compute_type=compute_type, download_root=model_dir)

2025-10-19 22:58:35 - whisperx.asr - INFO - No language specified, language will be detected for each audio file (increases inference time)
2025-10-19 22:58:35 - whisperx.vads.pyannote - INFO - Performing voice activity detection using Pyannote...


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint .venv/lib/python3.10/site-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.4.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.8.0+cu128. Bad things might happen unless you revert torch to 1.x.


/home/slava/Documents/projects/Hanzo/wisper_python31013/.venv/lib/python3.10/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


In [28]:
transcribasiotn_start_time = time.time()
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

2025-10-19 22:59:02 - whisperx.asr - INFO - Detected language: ru (0.99) in first 30s of audio
[{'text': ' Да. Доброе утро. Доброе утро. А время сколько-то сейчас? Шесть часов. Твою мать, я только уснула. А что ты делала-то? Что-то что-то. То на день, то так. Крутилась-крутилась. Никак. Вот так вот. Ей-то еще где была. В три часа ночи звонит. Ты доехала до дома?', 'start': 1.55, 'end': 31.182}, {'text': ' Я говорю, я уже давно доехала. Ну и все. И сон закончился. Что дома сидишь, приезжай в гости. Ну давай. Ну вот до трех я кое-как заснул. Как бы домой сейчас не вернулась.', 'start': 31.655, 'end': 57.49}, {'text': ' Ясно? Почему? Алло. Да, я слышал. Вот так вот. И такой вариант возможен. Хорошо услышал телефон, что звенит. Ага. Как самочувствие ваше? Ясно.', 'start': 63.245, 'end': 93.13}, {'text': ' Ясно. Ну ладно, пошла я вставать потихоньку. Давай. Позвонишь. Если на работе, то не возьму трубку. Если дома, значит, возьму. Давай.', 'start': 93.282, 'end': 115.912}]


In [29]:
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)


In [30]:
print(result["segments"]) # after alignment

print(f" Took time: {time.time() - start_time} seconds")
print(f" Transcribastion took time: {time.time() - transcribasiotn_start_time} seconds")

[{'start': 1.55, 'end': 3.692, 'text': ' Да.', 'words': [{'word': 'Да.', 'start': np.float64(1.55), 'end': np.float64(3.692), 'score': np.float64(0.859)}]}, {'start': 3.712, 'end': 5.574, 'text': 'Доброе утро.', 'words': [{'word': 'Доброе', 'start': np.float64(3.712), 'end': np.float64(4.073), 'score': np.float64(0.312)}, {'word': 'утро.', 'start': np.float64(4.193), 'end': np.float64(5.574), 'score': np.float64(0.695)}]}, {'start': 5.714, 'end': 8.277, 'text': 'Доброе утро.', 'words': [{'word': 'Доброе', 'start': np.float64(5.714), 'end': np.float64(6.155), 'score': np.float64(0.517)}, {'word': 'утро.', 'start': np.float64(6.215), 'end': np.float64(8.277), 'score': np.float64(0.726)}]}, {'start': 8.317, 'end': 10.279, 'text': 'А время сколько-то сейчас?', 'words': [{'word': 'А', 'start': np.float64(8.317), 'end': np.float64(8.357), 'score': np.float64(0.828)}, {'word': 'время', 'start': np.float64(8.397), 'end': np.float64(8.658), 'score': np.float64(0.971)}, {'word': 'сколько-то', 's

In [31]:
diarize_model = DiarizationPipeline('pyannote/speaker-diarization-3.1',
                                    use_auth_token=os.getenv("use_auth_token"),
                                    device=device)

2025-10-19 23:00:23 - whisperx.diarize - INFO - Loading diarization model: pyannote/speaker-diarization-3.1


/home/slava/Documents/projects/Hanzo/wisper_python31013/.venv/lib/python3.10/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


In [32]:
# add min/max number of speakers if known
diarize_segments = diarize_model(audio, min_speakers=2)
# diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)

/home/slava/Documents/projects/Hanzo/wisper_python31013/.venv/lib/python3.10/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


In [33]:
result = whisperx.assign_word_speakers(diarize_segments, result)

In [34]:
print("diarize_segments")
print(diarize_segments)
print("result")
print(result["segments"]) # segments are now assigned speaker IDs

diarize_segments
                              segment label     speaker       start  \
0   [ 00:00:01.583 -->  00:00:02.275]     A  SPEAKER_01    1.583469   
1   [ 00:00:02.646 -->  00:00:02.882]     B  SPEAKER_00    2.646594   
2   [ 00:00:03.152 -->  00:00:04.249]     C  SPEAKER_00    3.152844   
3   [ 00:00:04.317 -->  00:00:04.367]     D  SPEAKER_00    4.317219   
4   [ 00:00:05.616 -->  00:00:06.865]     E  SPEAKER_01    5.616594   
5   [ 00:00:06.409 -->  00:00:06.730]     F  SPEAKER_00    6.409719   
6   [ 00:00:08.232 -->  00:00:09.632]     G  SPEAKER_01    8.232219   
7   [ 00:00:10.139 -->  00:00:10.763]     H  SPEAKER_01   10.139094   
8   [ 00:00:12.214 -->  00:00:13.885]     I  SPEAKER_01   12.214719   
9   [ 00:00:14.948 -->  00:00:14.965]     J  SPEAKER_01   14.948469   
10  [ 00:00:14.965 -->  00:00:15.792]     K  SPEAKER_00   14.965344   
11  [ 00:00:17.479 -->  00:00:18.053]     L  SPEAKER_01   17.479719   
12  [ 00:00:18.559 -->  00:00:19.740]     M  SPEAKER_01   18

In [35]:
result

{'segments': [{'start': 1.55,
   'end': 3.692,
   'text': ' Да.',
   'words': [{'word': 'Да.',
     'start': np.float64(1.55),
     'end': np.float64(3.692),
     'score': np.float64(0.859),
     'speaker': 'SPEAKER_00'}],
   'speaker': 'SPEAKER_00'},
  {'start': 3.712,
   'end': 5.574,
   'text': 'Доброе утро.',
   'words': [{'word': 'Доброе',
     'start': np.float64(3.712),
     'end': np.float64(4.073),
     'score': np.float64(0.312),
     'speaker': 'SPEAKER_00'},
    {'word': 'утро.',
     'start': np.float64(4.193),
     'end': np.float64(5.574),
     'score': np.float64(0.695),
     'speaker': 'SPEAKER_00'}],
   'speaker': 'SPEAKER_00'},
  {'start': 5.714,
   'end': 8.277,
   'text': 'Доброе утро.',
   'words': [{'word': 'Доброе',
     'start': np.float64(5.714),
     'end': np.float64(6.155),
     'score': np.float64(0.517),
     'speaker': 'SPEAKER_01'},
    {'word': 'утро.',
     'start': np.float64(6.215),
     'end': np.float64(8.277),
     'score': np.float64(0.726),
   

In [36]:
type(result)

dict

In [37]:
result.get("segments")[0]

{'start': 1.55,
 'end': 3.692,
 'text': ' Да.',
 'words': [{'word': 'Да.',
   'start': np.float64(1.55),
   'end': np.float64(3.692),
   'score': np.float64(0.859),
   'speaker': 'SPEAKER_00'}],
 'speaker': 'SPEAKER_00'}

In [39]:
import json


In [42]:
import os

with open(os.path.join(os.getenv("upload_dir","./"), "diarized.json"), "w") as f:
    json.dump(result, f)

In [38]:
for s in result.get("segments"):
    #print(s)
    print(f" {s.get('speaker')} : {s.get('text')} ")


    #break

 SPEAKER_00 :  Да. 
 SPEAKER_00 : Доброе утро. 
 SPEAKER_01 : Доброе утро. 
 SPEAKER_01 : А время сколько-то сейчас? 
 SPEAKER_01 : Шесть часов. 
 SPEAKER_01 : Твою мать, я только уснула. 
 SPEAKER_00 : А что ты делала-то? 
 SPEAKER_01 : Что-то что-то. 
 SPEAKER_01 : То на день, то так. 
 SPEAKER_01 : Крутилась-крутилась. 
 SPEAKER_01 : Никак. 
 SPEAKER_01 : Вот так вот. 
 SPEAKER_01 : Ей-то еще где была. 
 SPEAKER_01 : В три часа ночи звонит. 
 SPEAKER_01 : Ты доехала до дома? 
 SPEAKER_01 :  Я говорю, я уже давно доехала. 
 SPEAKER_01 : Ну и все. 
 SPEAKER_01 : И сон закончился. 
 SPEAKER_01 : Что дома сидишь, приезжай в гости. 
 SPEAKER_01 : Ну давай. 
 SPEAKER_01 : Ну вот до трех я кое-как заснул. 
 SPEAKER_01 : Как бы домой сейчас не вернулась. 
 SPEAKER_01 :  Ясно? 
 SPEAKER_00 : Почему? 
 None : Алло. 
 SPEAKER_01 : Да, я слышал. 
 SPEAKER_01 : Вот так вот. 
 SPEAKER_01 : И такой вариант возможен. 
 SPEAKER_01 : Хорошо услышал телефон, что звенит. 
 SPEAKER_01 : Ага. 
 SPEAKER_0